In [1]:
#먼저 본인의 googledrive에 coco.zip 파일을 넣어줌(압축 안 푼 것 넣기! 압축 푼 것 넣지 마세요)
#coco.zip파일은 https://www.kaggle.com/datasets/s076923/pytorch-transformer에서 다운로드(wget http://images.cocodataset.org/zips/val2017.zip로 따로 받지 마세요)
#googledrive을 google colab에 마운트.
#압축을 풀어서 코랩에 train, val, annotations 파일을 생성.
!unzip /content/drive/MyDrive/CV_study/Coco_dataset/coco.zip

Archive:  /content/drive/MyDrive/CV_study/Coco_dataset/coco.zip
  inflating: annotations/train_annotations.json  
  inflating: annotations/val_annotations.json  
  inflating: train/000000000042.jpg  
  inflating: train/000000000074.jpg  
 extracting: train/000000000400.jpg  
  inflating: train/000000000599.jpg  
  inflating: train/000000000675.jpg  
 extracting: train/000000000711.jpg  
  inflating: train/000000001319.jpg  
 extracting: train/000000001398.jpg  
  inflating: train/000000002295.jpg  
 extracting: train/000000002477.jpg  
  inflating: train/000000002753.jpg  
  inflating: train/000000002988.jpg  
 extracting: train/000000003067.jpg  
 extracting: train/000000003711.jpg  
 extracting: train/000000004212.jpg  
 extracting: train/000000004678.jpg  
 extracting: train/000000005105.jpg  
 extracting: train/000000005577.jpg  
 extracting: train/000000005617.jpg  
 extracting: train/000000005804.jpg  
  inflating: train/000000006397.jpg  
  inflating: train/000000006589.jpg  
  

In [1]:
#os = Operating System, 여기선 운영체제에 맞게 경로 생성해줌
#torch = pytorch 사용, 파이토치는 인공신경망을 만드는데 필요한 다양한 기본 요소를 간단하고 직관적이며 안정적인 API로 제공함
#PIL = Python Imaging Library, 파이썬 이미지 처리 라이브러리
#pycocotools.coco = COCO 데이터 셋을 활용하기 위한 여러 유용한 함수들을 제공(e.g. getAnnIds, getCatIds etc)
#torch.utils.data = 파이토치의 데이터 불러오는 역할 수행

import os
import torch
from PIL import Image
from pycocotools.coco import COCO
from torch.utils.data import Dataset

#coco dataset 다루는 클래스
class COCODataset(Dataset):
    #어노테이션, image path 등 준비, transform은 데이터 변형 및 데이터 증강을 말하는데 데이터 변형 안함(transform = None)
    def __init__(self, root, train, transform=None):
        super().__init__()
        directory = "train" if train else "val"
        annotations = os.path.join(root, "annotations", f"{directory}_annotations.json")

        self.coco = COCO(annotations)
        self.iamge_path = os.path.join(root, directory)
        self.transform = transform

        self.categories = self._get_categories()
        self.data = self._load_data()

    #카테고리 설정
    def _get_categories(self):
        categories = {0: "background"}
        for category in self.coco.cats.values():
            categories[category["id"]] = category["name"]
        return categories

    #이미지 불러오기
    def _load_data(self):
        data = []
        for _id in self.coco.imgs:
            file_name = self.coco.loadImgs(_id)[0]["file_name"]
            image_path = os.path.join(self.iamge_path, file_name)
            image = Image.open(image_path).convert("RGB")

            boxes = []
            labels = []
            anns = self.coco.loadAnns(self.coco.getAnnIds(_id))

            #박스와 레이블 만들기
            for ann in anns:
                x, y, w, h = ann["bbox"]

                boxes.append([x, y, x + w, y + h])
                labels.append(ann["category_id"])

            #ground truth(supervised learning에서 정한 이미 정해놓은 박스)가 target로 나옴
            target = {
                "image_id": torch.LongTensor([_id]),
                "boxes": torch.FloatTensor(boxes),
                "labels": torch.LongTensor(labels)
            }
            data.append([image, target])
        return data

    #데이터 아이템(이미지, ground truth box)을 뽑기
    def __getitem__(self, index):
        image, target = self.data[index]
        if self.transform:
            image = self.transform(image)
        return image, target

    #데이터 크기 찾기
    def __len__(self):
        return len(self.data)

In [4]:
#torchvision = 파이토치에서 이미지 데이터의 전처리 및 데이터 증강을 위해 제공하는 모듈
from torchvision import transforms
from torch.utils.data import DataLoader

#배치를 만들기 위해 데이터들을 합쳐주는 함수(collate = 합치다)
#zip = 여러 개의 리스트나 튜플 타입을 합쳐 새로운 튜플(tuple) 타입으로 반환
def collator(batch):
    return tuple(zip(*batch))

#이미지를 변환시켜주기, 이미지를 텐서로 만듦(데이터 타입은 torch.float로 나오게 함)
#e.g. torch.FloatTensor([[1,2], [3,4]]) -> tensor([[1., 2.],[3., 4.]])
transform = transforms.Compose(
    [
        transforms.PILToTensor(),
        transforms.ConvertImageDtype(dtype=torch.float)
    ]
)

#validation data를 test_dataset으로 넣어줌
test_dataset = COCODataset("/content/val", train=False, transform=transform)

#train_dataloader과 test_dataloader의 batch_size 1로 설정해줌, 데이터 shuffle해놓음.??
train_dataloader = DataLoader(
    test_dataset, batch_size=1, shuffle=True, drop_last=True, collate_fn=collator
)
test_dataloader = DataLoader(
    test_dataset, batch_size=1, shuffle=True, drop_last=True, collate_fn=collator
)

loading annotations into memory...
Done (t=0.01s)
creating index...
index created!


In [5]:
#rpn = region proposal network
from torchvision import models
from torchvision import ops
from torchvision.models.detection import rpn
from torchvision.models.detection import FasterRCNN

#백본은 VGG16(Very Deep Convolutional Networks For Large-Scale Image Recognition)을 사용
#VGG16은 16개의 레이어로 구성(13개 convolution layers, 3개 fully connected layers, 완전신경망, 한층의 모든 뉴런이 다음 층의 모든 뉴런에 연결된 것)
#VGG16은 convolution filter를 3x3로 구성.
#VGG16의 아웃풋 채널은 512.??
#VGG 논문이해 -> https://medium.com/@msmapark2/vgg16-%EB%85%BC%EB%AC%B8-%EB%A6%AC%EB%B7%B0-very-deep-convolutional-networks-for-large-scale-image-recognition-6f748235242a
backbone = models.vgg16(weights="VGG16_Weights.IMAGENET1K_V1").features
backbone.out_channels = 512

#앵커박스 만들기 -> 책 p.513, 그림 9.6
anchor_generator = rpn.AnchorGenerator(
    sizes=((32, 64, 128, 256, 512),),
    aspect_ratios=((0.5, 1.0, 2.0),)
)

#관심영역풀링(ROI pooling) -> 고정된 크기의 특징 맵 얻음. 이 특징 맵을 완전 연결 계층에 전달해 물체의 클래스와 위치 예측.??
roi_pooler = ops.MultiScaleRoIAlign(
    featmap_names=["0"],
    output_size=(7, 7),
    sampling_ratio=2
)

#gpu 사용하면 cuda 아니면 cpu사용.
device = "cuda" if torch.cuda.is_available() else "cpu"

#우리가 만드는 FasterRCNN모델의 구조, num_classes는 background, cat, dog임.
model = FasterRCNN(
    backbone=backbone,
    num_classes=3,
    rpn_anchor_generator=anchor_generator,
    box_roi_pool=roi_pooler
).to(device)

Downloading: "https://download.pytorch.org/models/vgg16-397923af.pth" to /root/.cache/torch/hub/checkpoints/vgg16-397923af.pth
100%|██████████| 528M/528M [00:03<00:00, 159MB/s]


In [6]:
#optim은 최적화 알고리즘을 구현하는 패키지
from torch import optim

#requires_grad가 True이면 모델 파라미터들을 optimizer에서 업데이트해줌.
#optimizer에서 weight_decay를 하는 이유는 과적합을 막기위해. weight 너무 커져서 모델이 너무 복잡해지지 않도록 함(https://light-tree.tistory.com/216)
#weight decay는 L1 Regularization 또는 L2 Regularization 등이 쓰임.
#lr_schedular step size마다 gamma 비율로 lr를 감소시킨다(https://wikidocs.net/157282)
params = [p for p in model.parameters() if p.requires_grad]
optimizer = optim.SGD(params, lr=0.001, momentum=0.9, weight_decay=0.0005)
lr_scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.1)

In [2]:
#Python3, CPU, 고용량 RAM, 도서관 Public Wifi Free에서 약 3시간 소요
#모델 트레이닝
for epoch in range(5):
    cost = 0.0
    for idx, (images, targets) in enumerate(train_dataloader): # validation data가 181개, 그래서 181 iterations
        images = list(image.to(device) for image in images)
        targets = [{k: v.to(device) for k, v in t.items()} for t in targets]

        loss_dict = model(images, targets) # L_cls, L_reg?
        #print("loss_dict", loss_dict) 시간될 때 뭐 나오나 확인해보기??
        losses = sum(loss for loss in loss_dict.values())

        #한번의 학습이 완료되면(iteration 완료 시), gradients를 0으로 만들어줌
        optimizer.zero_grad()
        #역전파 적용, loss 함수 기록
        losses.backward()
        #최적화 알고리즘 적용하여 필요 시, 파라미터(weight, bias) 업데이트(Loss 함수 작아지면 업데이트)
        optimizer.step()

        cost += losses

    lr_scheduler.step()
    cost = cost / len(train_dataloader) # 181
    print(f"Epoch : {epoch+1:4d}, Cost : {cost:.3f}")

NameError: name 'train_dataloader' is not defined

In [9]:
#Python3, CPU, 고용량 RAM, 도서관 Public Wifi Free에서 11분 걸림
#matplotlib는 파이썬에서 데이터 시각화해주는 라이브러리
import numpy as np
from PIL import Image
from matplotlib import pyplot as plt
from torchvision.transforms.functional import to_pil_image


def draw_bbox(ax, box, text, color):
    #bounding box 그리기
    ax.add_patch(
        plt.Rectangle(
            xy=(box[0], box[1]),
            width=box[2] - box[0],
            height=box[3] - box[1],
            fill=False,
            edgecolor=color,
            linewidth=2,
        )
    )
    #bounding box의 annotation 그리기
    ax.annotate(
        text=text,
        xy=(box[0] - 5, box[1] - 5),
        color=color,
        weight="bold",
        fontsize=13,
    )

#IoU(Intersection over Union)의 threshold를 0.5로 설정 (p.516 참조)
threshold = 0.5
categories = test_dataset.categories
with torch.no_grad():
    model.eval()
    for images, targets in test_dataloader: # 181 iters
        images = [image.to(device) for image in images]
        outputs = model(images)

        boxes = outputs[0]["boxes"].to("cpu").numpy()
        labels = outputs[0]["labels"].to("cpu").numpy()
        scores = outputs[0]["scores"].to("cpu").numpy()

        #IoU에서 threshold가 0.5이상일 때만 boxes, labels, scores 기록, 나중에 빨간색 박스로 그림에서 나옴
        boxes = boxes[scores >= threshold].astype(np.int32)
        labels = labels[scores >= threshold]
        scores = scores[scores >= threshold]

        fig = plt.figure(figsize=(8, 8))
        ax = fig.add_subplot(1, 1, 1)
        plt.imshow(to_pil_image(images[0]))

        #우리 모델이 훈련한 box를 빨간색으로 나오게 함
        for box, label, score in zip(boxes, labels, scores):
            draw_bbox(ax, box, f"{categories[label]} - {score:.4f}", "red")

        tboxes = targets[0]["boxes"].numpy()
        tlabels = targets[0]["labels"].numpy()

        #ground truth는 파란색 box로 나오게 함.
        for box, label in zip(tboxes, tlabels):
            draw_bbox(ax, box, f"{categories[label]}", "blue")

        plt.show()

Output hidden; open in https://colab.research.google.com to view.

In [10]:
#Python3, CPU, 고용량 RAM, 도서관 Public Wifi Free에서 9분 걸림
import numpy as np
from pycocotools.cocoeval import COCOeval


with torch.no_grad():
  #모델을 evaluation함
    model.eval()
    coco_detections = []
    for images, targets in test_dataloader:
        images = [img.to(device) for img in images]
        outputs = model(images)

        #boxes 왜 세개임??
        for i in range(len(targets)):
            image_id = targets[i]["image_id"].data.cpu().numpy().tolist()[0]
            boxes = outputs[i]["boxes"].data.cpu().numpy()
            boxes[:, 2] = boxes[:, 2] - boxes[:, 0]
            boxes[:, 3] = boxes[:, 3] - boxes[:, 1]
            scores = outputs[i]["scores"].data.cpu().numpy()
            labels = outputs[i]["labels"].data.cpu().numpy()

            #boxes 왜 세개임??
            for instance_id in range(len(boxes)):
                box = boxes[instance_id, :].tolist()
                prediction = np.array(
                    [
                        image_id,
                        box[0],
                        box[1],
                        box[2],
                        box[3],
                        float(scores[instance_id]),
                        int(labels[instance_id]),
                    ]
                )
                coco_detections.append(prediction)

    coco_detections = np.asarray(coco_detections)
    coco_gt = test_dataloader.dataset.coco
    coco_dt = coco_gt.loadRes(coco_detections)
    coco_evaluator = COCOeval(coco_gt, coco_dt, iouType="bbox")
    coco_evaluator.evaluate()
    coco_evaluator.accumulate()
    coco_evaluator.summarize()

Loading and preparing results...
Converting ndarray to lists...
(2354, 7)
0/2354
DONE (t=0.01s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=0.16s).
Accumulating evaluation results...
DONE (t=0.08s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.187
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 0.482
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 0.074
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = 0.026
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = 0.247
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=100 ] = 0.185
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=  1 ] = 0.308
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets= 10 ] = 0.420
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.422
 Average Recall     